# Imports

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
plt.style.use("fast")
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
import plotly.subplots as sp
from plotly.subplots import make_subplots
import plotly.graph_objs as go

import random
import statistics

from sklearn.metrics import (
    accuracy_score,
    f1_score,
    classification_report,
    confusion_matrix,
    roc_auc_score,
)

from sklearn.metrics import roc_curve, auc
import matplotlib.cm as cm
from scipy import interp

!pip install scikit-plot -q
import scikitplot

from sklearn.preprocessing import LabelBinarizer

from sklearn.model_selection import train_test_split

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score

from sklearn.tree import DecisionTreeClassifier, plot_tree

from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold

In [ ]:
print(pd.__version__)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Dataset encoded but not scaled.

In [ ]:
datasetPath = '/content/drive/MyDrive/UNIPI/DM1/DATASET: Spotify/DM1 - Project/files/dm1_df_understanding_NOTSCALED.csv'
data = pd.read_csv(datasetPath)

data.head(3)

In order to easily map object variables to the integers with which they were replaced (we are talking about `name`, `artists`, `album_name`, and `genre`), we decided to save to disk a dictionary of dictionaries, containing each integer associated with the original string. This way we can see what `genre` each integer in the genre column is associated with.

In [ ]:
import pickle

# Carica il dizionario inverso da disco
with open('/content/drive/MyDrive/UNIPI/DM1/DATASET: Spotify/DM1 - Project/files/inverse_column_dicts.pkl', 'rb') as f:
    loaded_inverse_column_dicts = pickle.load(f)

genre_dict = loaded_inverse_column_dicts['genre']
genre_dict

In [ ]:
df = data.copy(deep=True)

In [ ]:
df.info()

In [ ]:
df.drop(['name','artists', 'album_name'], axis=1, inplace=True)

In [ ]:
# One-Hot Encoding 'key'
df = pd.get_dummies(df, columns=['key'])

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
# sns.countplot(data=df, x=df['genre'])

### TEST SET

In [ ]:
testSetPath = '/content/drive/MyDrive/UNIPI/DM1/DATASET: Spotify/DM1 - Project/files/test.csv'
test_df = pd.read_csv(testSetPath)

test_df.head(3)

In [ ]:
# Pre-Processing
test_df.drop(['name','artists', 'album_name','time_signature','popularity_confidence','features_duration_ms','n_beats'], axis=1, inplace=True)
test_df['duration_min'] = test_df['duration_ms'] / 60000
test_df.drop('duration_ms', axis=1, inplace=True)
test_df['explicit'] = [int(x) for x in test_df['explicit']] # bool->int (binary)
test_df['key'] = [int(x) for x in test_df['key']] # float -> int

data_modeNoNull = test_df.dropna(subset=['mode'], axis=0)
mode_1_records = test_df[test_df['mode'] == 1].shape[0]
mode_0_records = test_df[test_df['mode'] == 0].shape[0]
percent_1 = mode_1_records / data_modeNoNull.shape[0]
percent_0 = mode_0_records / data_modeNoNull.shape[0]
null_count_mode = test_df['mode'].isnull().sum()
count_1 = int(null_count_mode * (percent_1 / 100))
count_0 = null_count_mode - count_1
random_values = np.random.choice([1, 0], size=null_count_mode, p=[percent_1, percent_0])
test_df.loc[test_df['mode'].isnull(), 'mode'] = random_values
test_df['mode'] = [int(x) for x in test_df['mode']] # float->int (binary)

test_df.info()

In [ ]:
# Ottieni i generi nel training set
train_genres = set(genre_dict.values())

# Ottieni i generi unici nel test set
test_genres = set(test_df['genre'].unique())

# Confronta i generi
common_genres = train_genres.intersection(test_genres)
train_only_genres = train_genres.difference(test_genres)
test_only_genres = test_genres.difference(train_genres)

print("Generi comuni:", common_genres)
print("Generi solo nel training set:", train_only_genres)
print("Generi solo nel test set:", test_only_genres)

In [ ]:
# Inverti il dizionario
genre_dict_test = {v: k for k, v in genre_dict.items()}
# Ora mappa i generi del test set agli interi corrispondenti
test_df['genre'] = test_df['genre'].map(genre_dict_test)

In [ ]:
test_df = pd.get_dummies(test_df, columns=['key'])
print(test_df.shape, df.shape)
test_df.head()

# TARGET: `genre`

In [ ]:
# Drop the 'genre' column (target) from the DataFrame
df_without_target = df.drop('genre', axis=1)

# Convert the DataFrame without the target column to a NumPy array
X = df_without_target.values

# Save column names
columns = df_without_target.columns.tolist()

# Convert target column to NumPy array
y = np.array(df['genre'])

# Check unique values and their counts
np.unique(y, return_counts=True)

The stratify parameter is used to ensure that the division of training and test data maintains the same class distribution as in the original data set.

In [ ]:
# PARTITIONING

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.4, stratify=y, random_state=0
)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

In [ ]:
from sklearn.preprocessing import StandardScaler

norm = StandardScaler()
norm.fit(X_train)

X_train_norm = norm.transform(X_train)
X_test_norm = norm.transform(X_test)

## KNN

In [ ]:
# Base model with default parameters
base_knn = KNeighborsClassifier()
base_knn.fit(X_train_norm, y_train)

base_y_test_pred = base_knn.predict(X_test_norm)

print(f"Accuracy: {accuracy_score(y_test, base_y_test_pred):.2f}")

average="micro": This calculates the metrics globally by counting the total true positives, false negatives, and false positives1. It aggregates the contributions of all classes to compute the average metric2. In other words, it computes the F1 score across all classes, which is useful if you have class imbalance2. This means that micro F1 score gives equal importance to each observation

In [ ]:
print(f"F1: {f1_score(y_test, base_y_test_pred, average='micro'):.2f}")

#### Hyperparameters Tuning

This code uses cross-validation to evaluate the performance of the model. Instead of dividing the data into a training set and a test set, cross-validation divides the data into `k` groups or "folds." The model is then trained on `k-1` fold and tested on the remaining fold. This process is repeated `k` times, with each fold used once as a test set. The mean and standard deviation of accuracy over all folds are then calculated and plotted in a graph for each value of `n_neighbors`.

We set `k=5`.

The vertical bars in the graph generated by the second code represent the standard deviation of model accuracy for each value of n_neighbors during cross-validation. These bars are called "error bars."


The standard deviation is a measure of the variability or dispersion of the data. In this case, it indicates how much the accuracy of the model varies between different folds of the cross-validation. If the error bar for a certain value of n_neighbors is large, it means that the accuracy of the model varies greatly among different folds. If the error bar is small, it means that the accuracy of the model is about the same for all folds.


Error bars help to understand how reliable the average accuracy estimate is. If the error bars are small, we can be more confident that the average accuracy accurately represents the performance of the model. If the error bars are large, the average accuracy may not be a good estimate of model performance.

In [ ]:
%%time
# 2/3 min

n_neighbors = range(1,100)
avg_scores = list()
std_scores = list()
plt.figure(figsize=(15,5))

for n in n_neighbors:
    clf = KNeighborsClassifier(n_neighbors=n)
    scores = cross_val_score(clf, X_train_norm, y_train, cv=5)
    avg_scores.append(np.mean(scores))
    std_scores.append(np.std(scores))

max_score_index = np.argmax(avg_scores)
max_n_neighbors = n_neighbors[max_score_index]
max_score = avg_scores[max_score_index]

#plt.plot(avg_scores)
plt.errorbar(range(len(n_neighbors)), y=avg_scores, yerr=std_scores, marker='o', color='green')
plt.xticks(range(len(n_neighbors)), n_neighbors)
plt.xlabel("n_neighbors")
plt.ylabel("accuracy")

plt.text(0.1, 0.9, f'Max accuracy at n_neighbors={max_n_neighbors}',
         transform=plt.gca().transAxes, bbox=dict(facecolor='teal', alpha=0.5))

plt.show()

In [ ]:
knn = KNeighborsClassifier(n_neighbors=max_n_neighbors)
knn.fit(X_train_norm, y_train)
y_test_pred = knn.predict(X_test_norm)
print(f"Accuracy: {accuracy_score(y_test, y_test_pred):.2f}")

Above 10/12 neighbors the accuracy stabilizes around a value of about 0.43 (for the chosen range of n_neighbors between 1 and 50 the maximum accuracy, 0.44, is reached for n_neighbors=14). Selecting a range of neighbors between 10 and 15 we can evaluate othe hyperparameters using a gridSearch.

In [ ]:
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

In [ ]:
%%time
# ~14min

param_grid = {
    "n_neighbors": np.arange(10, 20),
    "weights": ["uniform", "distance"],
    "metric": ["euclidean", "cityblock"],
}

grid = GridSearchCV(
    KNeighborsClassifier(),
    param_grid=param_grid,
    cv=RepeatedStratifiedKFold(random_state=0),
    n_jobs=-1,
    refit=True,
    verbose=0
)

grid.fit(X_train_norm, y_train)
best_knn = grid.best_estimator_

print(grid.best_params_, grid.best_score_)

best_y_test_pred = best_knn.predict(X_test_norm)
print(f"Accuracy: {accuracy_score(y_test, best_y_test_pred):.2f}")
print(f"F1: {f1_score(y_test, best_y_test_pred, average='micro'):.2f}")

The configuration `{'metric': 'cityblock', 'n_neighbors': 14, 'weights': 'distance'}` performs best, with a final accuracy of about 0.48.

In [ ]:
'''plt.figure(figsize=(20,20))
cf = confusion_matrix(y_test, best_y_test_pred)
sns.heatmap(cf, annot=True, cmap="Greens")
plt.xlabel("True")
plt.ylabel("Predicted")
plt.show()'''

In [ ]:
results = pd.DataFrame(grid.cv_results_)
results["metric_weight"] = results["param_metric"] + ", " + results["param_weights"]
sns.lineplot(
    data=results, x="param_n_neighbors", y="mean_test_score", hue="metric_weight"
)

For multiple classes we can choose one as positive and all the others as negative and compute precision, recall, F-score. Repeating the process for all classes we can obtain an average accuracy.

#### Performance evaluation

In [ ]:
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelBinarizer

# Binarize the output

'''
LabelBinarizer is used to transform categorical labels into a binary format,
often referred to as one-hot encoding. It transforms categorical labels into a binary
matrix. Each label is replaced with a row containing 0s and a single 1 at the corresponding
class column, indicating the presence of that class.
'''

lb = LabelBinarizer()
y_test_bin = lb.fit_transform(y_test)
best_y_test_pred_bin = lb.transform(best_y_test_pred)

# Compute precision, recall, F-score for each class
report = classification_report(y_test_bin, best_y_test_pred_bin, output_dict=True)

# Compute average accuracy
avg_accuracy = sum([report[str(i)]['f1-score'] for i in range(len(lb.classes_))]) / len(lb.classes_)

print(f"Average Accuracy: {avg_accuracy:.2f}")

In [ ]:
def plot_roc(y_test, y_score, n_classes):

    # Compute ROC curve and ROC area for each class
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(y_test_bin[:, i], y_score[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])

    # Compute micro-average ROC curve and ROC area
    fpr["micro"], tpr["micro"], _ = roc_curve(y_test_bin.ravel(), y_score.ravel())
    roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

    # Plot all ROC curves
    plt.figure(figsize=(8,8))
    plt.plot(fpr["micro"], tpr["micro"],
             label='micro-average ROC curve (area = {0:0.2f})'
                   ''.format(roc_auc["micro"]),
             color='green', linestyle=':', linewidth=8)

    colors = cm.Greys(np.linspace(0, 1, n_classes))
    for i, color in zip(range(n_classes), colors):
        plt.plot(fpr[i], tpr[i], color=color, lw=2,
             label='ROC curve of class {0} (area = {1:0.2f})'
             ''.format(i, roc_auc[i]))

    plt.plot([0, 1], [0, 1], 'k--', lw=2)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic to Multi-Class')
    plt.legend(loc="lower right")
    plt.show()

In [ ]:
from sklearn.metrics import roc_curve, auc
import matplotlib.cm as cm
from scipy import interp

n_classes = len(df['genre'].unique())
best_y_test_pred_proba = best_knn.predict_proba(X_test_norm)
best_y_test_pred_labels = best_y_test_pred_proba.argmax(axis=1)

print(f"F1:{f1_score(y_test, best_y_test_pred_labels, labels=[1], average='micro'):.2f}")

plot_roc(y_test, best_y_test_pred_proba, n_classes=n_classes)
plt.show()

In [ ]:
from sklearn.metrics import precision_recall_curve, average_precision_score
import matplotlib.cm as cm

def plot_precision_recall(y_test, y_score, n_classes):

    # Compute Precision-Recall curve and area for each class
    precision = dict()
    recall = dict()
    average_precision = dict()
    for i in range(n_classes):
        precision[i], recall[i], _ = precision_recall_curve(y_test_bin[:, i], y_score[:, i])
        average_precision[i] = average_precision_score(y_test_bin[:, i], y_score[:, i])

    # Compute micro-average Precision-Recall curve and area
    precision["micro"], recall["micro"], _ = precision_recall_curve(y_test_bin.ravel(), y_score.ravel())
    average_precision["micro"] = average_precision_score(y_test_bin, y_score, average="micro")

    # Plot all Precision-Recall curves
    plt.figure(figsize=(8,8))
    plt.plot(recall["micro"], precision["micro"],
             label='micro-average Precision-recall curve (area = {0:0.2f})'
                   ''.format(average_precision["micro"]),
             color='green', linestyle=':', linewidth=8)

    colors = cm.Greys(np.linspace(0, 1, n_classes))
    for i, color in zip(range(n_classes), colors):
        plt.plot(recall[i], precision[i], color=color, lw=2,
             label='Precision-recall curve of class {0} (area = {1:0.2f})'
             ''.format(i, average_precision[i]))

    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('Precision-Recall curve to Multi-Class')
    plt.legend(loc="lower right")
    plt.show()

plot_precision_recall(y_test, best_y_test_pred_proba, n_classes=n_classes)
plt.show()

#### Repeated Holdout

For each iteration of the loop, we are creating a new instance of the best model (`rh_clf = grid.best_estimator_`) before training it on the training data. This should ensure that there is no information from the test set that is used when training the model.

In [ ]:
%%time
# 3min

N = 100
err = 0

for i in range(N):
    X_rh_train, X_rh_test, y_rh_train, y_rh_test = train_test_split(X, y, test_size=0.4, stratify=y)

    # normalize train set
    norm.fit(X_rh_train)
    X_rh_train_norm = norm.transform(X_rh_train)
    X_rh_test_norm = norm.transform(X_rh_test)

    rh_clf = grid.best_estimator_
    rh_clf.fit(X_rh_train_norm, y_rh_train)

    # computing error
    acc = rh_clf.score(X_rh_test_norm, y_rh_test)
    err += 1 - acc

print(f"Overall error estimate: {err/N:.2f}")

In [ ]:
print(f"HO Accuracy: {acc}")

The total error we got is about 0.52. This means that, on average, the model made errors 52% of the time during 100 repeated holdout iterations. In other words, the model misclassified 52% of the instances in the test set.

#### Cross-Validation

In [ ]:
from sklearn.model_selection import cross_val_score
k = 10

scores = cross_val_score(best_knn, X_train_norm, y_train, cv=k)
print(f"Overall error estimate: {1 - scores.mean():.2f}")
print('Accuracy: %0.4f (+/- %0.2f)' % (scores.mean(), scores.std()))

### TEST

In [ ]:
# Drop the 'genre' column (target) from the DataFrame
df_without_target_EXT = test_df.drop('genre', axis=1)

# Convert the DataFrame without the target column to a NumPy array
X_testEXT = df_without_target_EXT.values

# Convert target column to NumPy array
y_ext = np.array(test_df['genre'])

X_testEXT_norm = norm.transform(X_testEXT)

scores_EXT = cross_val_score(best_knn, X_testEXT_norm, y_ext, cv=10)
print('RESULTS ON EXTERNAL TEST SET:')
print(f"Overall error estimate: {1 - scores_EXT.mean():.2f}")
print('Accuracy: %0.4f (+/- %0.2f)' % (scores_EXT.mean(), scores_EXT.std()))

## Naive Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB, CategoricalNB, BernoulliNB
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

In [ ]:
# GAUSSIAN_NB
'''
Utilizzo solo variabili continue
'''

from sklearn.preprocessing import label_binarize

nb_cont_features = [
    'danceability', 'energy', 'loudness', 'speechiness', 'acousticness',
    'instrumentalness', 'liveness', 'valence', 'tempo', 'n_bars', 'duration_min'
]

X_nbg = df[nb_cont_features].values

X_train_gauss, X_test_gauss, y_train_gauss, y_test_gauss = train_test_split(
    X_nbg, y, test_size=0.3, stratify=y, random_state=0
)

nbg = GaussianNB()
nbg.fit(X_train_gauss, y_train_gauss)

y_pred_nbg = nbg.predict(X_test_gauss)

print(classification_report(y_test_gauss, y_pred_nbg))
print(roc_auc_score(y_test_gauss, nbg.predict_proba(X_test_gauss), multi_class="ovr", average="micro"))
scores_nbg = cross_val_score(nbg, X_train_gauss, y_train_gauss, cv=k)
print(f"Overall error estimate: {1 - scores_nbg.mean():.2f}")
print('Accuracy: %0.4f (+/- %0.2f)' % (scores_nbg.mean(), scores_nbg.std()))

In [ ]:
# CATEGORICAL_NB

non_cat_columns = [
    'danceability', 'energy', 'loudness', 'speechiness', 'acousticness',
    'instrumentalness', 'liveness', 'valence', 'tempo', 'n_bars', 'duration_min'
]

X_noncat = df[non_cat_columns].values

X_train_noncat, X_test_noncat, y_train_noncat, y_test_noncat = train_test_split(
    X_noncat, y, test_size=0.3, stratify=y, random_state=0
)

X_train_cat = list()
for column_idx in range(X_train_noncat.shape[1]):
    X_train_cat.append(pd.qcut(X_train_noncat[:, column_idx], q=4, labels=False, duplicates='drop'))
X_train_cat = np.array(X_train_cat).T

X_test_cat = list()
for column_idx in range(X_test_noncat.shape[1]):
    X_test_cat.append(pd.qcut(X_test_noncat[:, column_idx], q=4, labels=False, duplicates='drop'))
X_test_cat = np.array(X_test_cat).T

print(X_train_cat.shape, X_test_cat.shape)

nbc = CategoricalNB()
nbc.fit(X_train_cat, y_train_noncat)

y_pred_nbc = nbc.predict(X_test_cat)

print(classification_report(y_test_noncat, y_pred_nbc))
print('ROC/AUC score: ',roc_auc_score(y_test_noncat, nbc.predict_proba(X_test_cat), multi_class="ovr", average="micro"))
scores_nbc = cross_val_score(nbc, X_train_cat, y_train_noncat, cv=10)
print(f"Overall error estimate: {1 - scores_nbc.mean():.2f}")
print('Accuracy: %0.4f (+/- %0.2f)' % (scores_nbc.mean(), scores_nbc.std()))

## Decision Trees

In [ ]:
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    classification_report,
    confusion_matrix,
    roc_auc_score,
)

In [ ]:
%%time
# 5min

# BASE MODEL
base_dt = DecisionTreeClassifier()

base_dt.fit(X_train, y_train)

plt.figure(figsize=(20, 4), dpi=300)
plot_tree(base_dt, feature_names=columns, filled=True)
plt.show()

y_train_pred = base_dt.predict(X_train)
y_test_pred = base_dt.predict(X_test)

print('Train Accuracy %s' % accuracy_score(y_train, y_train_pred))
print('Train F1-score %s' % f1_score(y_train, y_train_pred, average=None))
print()

print('Test Accuracy %s' % accuracy_score(y_test, y_test_pred))
print('Test F1-score %s' % f1_score(y_test, y_test_pred, average=None))

print(classification_report(y_test, y_test_pred))

cf = confusion_matrix(y_test, y_test_pred)
sns.heatmap(cf, annot=True, cmap="Greens")
plt.xlabel("True")
plt.ylabel("Predicted")
plt.show()

zipped = zip(columns, base_dt.feature_importances_)
zipped = sorted(zipped, key=lambda x: x[1], reverse=True)
for col, imp in zipped:
    print(col, imp)

print(roc_auc_score(y_test, base_dt.predict_proba(X_test), multi_class="ovr", average="micro"))
scores_baseDT = cross_val_score(base_dt, X_train, y_train, cv=10)
print(f"Overall error estimate: {1 - scores_baseDT.mean():.2f}")
print('Accuracy: %0.4f (+/- %0.2f)' % (scores_baseDT.mean(), scores_baseDT.std()))

### Hyperparameters Tuning

In [ ]:
'''
max_depth int, default=None

The maximum depth of the tree. If None, then nodes are expanded
until all leaves are pure or until all leaves contain less than min_samples_split samples.
'''

def depth_param_graph(interval, train_X, train_y, cv):
  max_depths = interval
  avg_scores = list()
  std_scores = list()
  scores = list()

  for max_depth in max_depths:
      dt = DecisionTreeClassifier(max_depth=max_depth)
      cross_val_scores = cross_val_score(dt, train_X, train_y, cv=cv)
      avg_scores.append(np.mean(cross_val_scores))
      std_scores.append(np.std(cross_val_scores))
      avg_score = np.mean(cross_val_scores)
      std_score = np.std(cross_val_scores)
      scores.append((max_depth, avg_score, std_score))

  scores.sort(key=lambda x: x[1], reverse=True)
  range_depth = sorted([t[0] for t in scores[:5]])
  print(range_depth)

  plt.figure(figsize=(25,5))
  plt.errorbar(range(len(max_depths)), y=avg_scores, yerr=std_scores, marker='o')
  plt.xticks(range(len(max_depths)), max_depths)
  plt.xlabel("max_depths")
  plt.ylabel("accuracy")
  plt.show()

  return range_depth

range_depth = depth_param_graph(interval=range(1, 51, 1), train_X=X_train, train_y=y_train, cv=5)

In [ ]:
'''
min_samples_split int or float, default=2

The minimum number of samples required to split an internal node:

- If int, then consider min_samples_split as the minimum number.
- If float, then min_samples_split is a fraction and ceil(min_samples_split * n_samples)
  are the minimum number of samples for each split.
'''

def split_param_graph(interval, train_X, train_y, cv):
  min_samples_splits = interval
  avg_scores = list()
  std_scores = list()
  scores = list()

  for min_samples_split in min_samples_splits:
     dt = DecisionTreeClassifier(min_samples_split=min_samples_split)
     cross_val_scores = cross_val_score(dt, train_X, train_y, cv=cv)
     avg_scores.append(np.mean(cross_val_scores))
     std_scores.append(np.std(cross_val_scores))
     avg_score = np.mean(cross_val_scores)
     std_score = np.std(cross_val_scores)
     scores.append((min_samples_split, avg_score, std_score))

  scores.sort(key=lambda x: x[1], reverse=True)
  range_split = sorted([t[0] for t in scores[:5]])
  print(range_split)

  plt.figure(figsize=(25,5))
  plt.errorbar(range(len(min_samples_splits)), y=avg_scores, yerr=std_scores, marker='o')
  plt.xticks(range(len(min_samples_splits)), min_samples_splits)
  plt.xlabel("min_samples_split")
  plt.ylabel("accuracy")
  plt.show()

  return range_split

range_split = split_param_graph(interval=range(2, 102, 2), train_X=X_train, train_y=y_train, cv=5)

In [ ]:
'''
min_samples_leaf int or float, default=1

The minimum number of samples required to be at a leaf node. A split point at any depth
will only be considered if it leaves at least min_samples_leaf training samples in each of
the left and right branches. This may have the effect of smoothing the model, especially in regression.

- If int, then consider min_samples_leaf as the minimum number.
- If float, then min_samples_leaf is a fraction and ceil(min_samples_leaf * n_samples) are the minimum number of samples for each node.
'''

def leaf_param_graph(interval, train_X, train_y, cv):
  min_samples_leafs = interval
  avg_scores = list()
  std_scores = list()
  scores = list()

  for min_samples_leaf in min_samples_leafs:
     dt = DecisionTreeClassifier(min_samples_leaf=min_samples_leaf)
     cross_val_scores = cross_val_score(dt, train_X, train_y, cv=cv)
     avg_scores.append(np.mean(cross_val_scores))
     std_scores.append(np.std(cross_val_scores))
     avg_score = np.mean(cross_val_scores)
     std_score = np.std(cross_val_scores)
     scores.append((min_samples_leaf, avg_score, std_score))

  scores.sort(key=lambda x: x[1], reverse=True)
  range_leaf = sorted([t[0] for t in scores[:5]])
  print(range_leaf)

  plt.figure(figsize=(25,5))
  plt.errorbar(range(len(min_samples_leafs)), y=avg_scores, yerr=std_scores, marker='o')
  plt.xticks(range(len(min_samples_leafs)), min_samples_leafs)
  plt.xlabel("min_samples_leaf")
  plt.ylabel("accuracy")
  plt.show()

  return range_leaf

range_leaf = leaf_param_graph(interval=range(1, 101, 1), train_X=X_train, train_y=y_train, cv=5)

### GridSearch

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold

In [ ]:
%%time
# 35min

# TODO: implement tqdm

param_list = { # based on graphs above
    'max_depth': range_depth,
    'min_samples_split': range_split,
    'min_samples_leaf': range_leaf,
    'criterion': ['gini', 'entropy'],
    'splitter':['best','random']
}

random_search = RandomizedSearchCV(
    DecisionTreeClassifier(),
    param_distributions=param_list,
    cv=RepeatedStratifiedKFold(random_state=0),
    n_jobs=-1, # number of jobs to run in parallel, -1 means using all processors
    refit=True, # allows using predict directly with this estimator
    n_iter=500 # the total size of your parameter space is the product of the number of values for each parameter; n_iter <= size (now 500)
    #verbose=2
)

random_search.fit(X_train, y_train)
dt = random_search.best_estimator_

In [ ]:
print(random_search.best_params_, random_search.best_score_)
y_train_pred = dt.predict(X_train)
y_test_pred = dt.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_test_pred))

print('Train Accuracy %s' % accuracy_score(y_train, y_train_pred))
print('Train F1-score %s' % f1_score(y_train, y_train_pred, average=None))
print()

print('Test Accuracy %s' % accuracy_score(y_test, y_test_pred))
print('Test F1-score %s' % f1_score(y_test, y_test_pred, average=None))

print(classification_report(y_test, y_test_pred))

In [ ]:
results = pd.DataFrame(random_search.cv_results_)

fig, axs = plt.subplots(2)

sns.lineplot(data=results, x="param_max_depth", y="mean_test_score", ax=axs[0])
axs[0].set_title('max_depth')

sns.lineplot(data=results, x="param_min_samples_leaf", y="mean_test_score", ax=axs[1])
axs[1].set_title('min_samples_leaf')

plt.tight_layout()
plt.show()

In [ ]:
zipped = zip(columns, dt.feature_importances_)
zipped = sorted(zipped, key=lambda x: x[1], reverse=True)
for col, imp in zipped:
    print(col, imp)

In [ ]:
plt.figure(figsize=(20, 4), dpi=300)
plot_tree(dt, feature_names=columns, filled=True)
plt.show()

In [ ]:
type(y)

In [ ]:
import matplotlib.font_manager as fm
fm.findSystemFonts(fontpaths=None, fontext='ttf')

In [ ]:
%%time
# 6min

import dtreeviz
import warnings
warnings.filterwarnings('ignore')

# Set the default font to one that is available on your system
plt.rcParams['font.family'] = 'LiberationMono-Regular'  # Example font

# Now you can create your visualization with dtreeviz
viz = dtreeviz.model(dt, X_train, y_train,
                     feature_names=columns,
                     class_names=list(np.unique(y)))

viz.view()

In [ ]:
viz.view().save("decision_tree_genre.svg")

In [ ]:
dt.tree_.children_left[6] = -1
dt.tree_.children_right[6]  = -1

viz2 = dtreeviz.model(dt, X_train, y_train,
                     feature_names=columns,
                     class_names=list(np.unique(y)))

viz2.view().save("decision_tree_genre_pruned.svg")

In [ ]:
viz2.view()

In [ ]:
from sklearn import tree
import graphviz

dot_data = tree.export_graphviz(dt, out_file=None,
                    feature_names=columns,
                    class_names=[str(i) for i in np.unique(y)],
                    filled=True, rounded=True,
                    special_characters=True)
graph = graphviz.Source(dot_data)
graph

In [ ]:
dot_data = tree.export_graphviz(dt, out_file=None,
                    feature_names=columns,
                    class_names=[str(i) for i in np.unique(y)],
                    filled=True, rounded=True,
                    special_characters=True, max_depth=2, rotate=True)
graph = graphviz.Source(dot_data)
graph

In [ ]:
graph.render("decision_tree_genre_graphviz", format="png")

In [ ]:
!pip install -q treeinterpreter
from treeinterpreter import treeinterpreter as ti

# Use TreeInterpreter to interpret the model's predictions
prediction, bias, contributions = ti.predict(dt, X_test)

# Print the results for the first instance in the test set
print("Prediction:", prediction[0])
print("Bias (trainset prior):", bias[0])
print("Feature contributions:")
for c, feature in zip(contributions[0], columns):
    print(feature, c)

The TreeInterpreter results provided can be interpreted as follows:

- Prediction: This is the predicted probability distribution over the classes for a given instance. In our case, the model predicts that the instance belongs to the third class with a probability of 0.34375, the sixth class with a probability of 0.15625, and so on.

- Bias (trainset prior): This is the prior probability distribution over the classes, which is typically the distribution of the classes in the training set. In our case, all classes have a prior probability of 0.05.

- Feature contributions: This shows how much each feature contributes to the shift from the prior probability to the predicted probability for each class. The contribution of a feature can be positive (increasing the probability of a class), negative (decreasing the probability), or zero (no effect).

For example, the `popularity` feature increases the probability of the third class by 0.08729017 and decreases the probability of the second class by 0.07425332. Similarly, the `danceability` feature increases the probability of the third class by 0.320599022 and decreases the probability of the tenth class by 0.132828714.

The sum of the bias and the feature contributions for each class should be equal to the predicted probability for that class.

### ccp_aplhas

Minimal cost complexity pruning recursively finds the node with the “weakest link”. The weakest link is characterized by an effective alpha, where the nodes with the smallest effective alpha are pruned first. <br>
Sklearn decision tree offers a function that returns the effective alphas and the corresponding total leaf impurities at each step of the pruning process. As alpha increases, more of the tree is pruned, which increases the total impurity of its leaves.

This part of the code computes the cost complexity pruning path associated with the decision tree on the training data. The cost_complexity_pruning_path function returns the effective alphas (the complexity parameter used for Minimal Cost-Complexity Pruning) and the corresponding total leaf impurities at each step of the pruning process. The total impurity of a tree is the sum of the impurity of its leaves.

Then, it plots the total impurity of leaves versus effective alpha for the training set. This helps in visualizing how the total impurity decreases with increasing alpha.

In [ ]:
path = dt.cost_complexity_pruning_path(X_train, y_train)
ccp_alphas, impurities = path.ccp_alphas, path.impurities

fig, ax = plt.subplots()
ax.plot(ccp_alphas[:-1], impurities[:-1], marker="o", drawstyle="steps-post")
ax.set_xlabel("effective alpha")
ax.set_ylabel("total impurity of leaves")
ax.set_title("Total Impurity vs effective alpha for training set")

For each effective alpha, a Decision Tree Classifier is trained with that alpha and the best parameters found by random_search. All the classifiers are stored in the clfs list.

In [ ]:
clfs = []
for ccp_alpha in ccp_alphas:
    clf = DecisionTreeClassifier(random_state=0, ccp_alpha=ccp_alpha, **random_search.best_params_)
    clf.fit(X_train, y_train)
    clfs.append(clf)
print(
    "Number of nodes in the last tree is: {} with ccp_alpha: {}".format(
        clfs[-1].tree_.node_count, ccp_alphas[-1]
    )
)

The code plots the number of nodes and the depth of the tree as a function of alpha. This helps in understanding how the complexity of the tree decreases with increasing alpha.

In [ ]:
clfs = clfs[:-1]
ccp_alphas = ccp_alphas[:-1]

node_counts = [clf.tree_.node_count for clf in clfs]
depth = [clf.tree_.max_depth for clf in clfs]
fig, ax = plt.subplots(2, 1)
ax[0].plot(ccp_alphas, node_counts, marker="o", drawstyle="steps-post")
ax[0].set_xlabel("alpha")
ax[0].set_ylabel("number of nodes")
ax[0].set_title("Number of nodes vs alpha")
ax[1].plot(ccp_alphas, depth, marker="o", drawstyle="steps-post")
ax[1].set_xlabel("alpha")
ax[1].set_ylabel("depth of tree")
ax[1].set_title("Depth vs alpha")
fig.tight_layout()

For each classifier in clfs, the training and testing accuracies are computed and plotted against alpha. This helps in understanding how the model’s performance varies with its complexity.

In [ ]:
train_scores = [clf.score(X_train, y_train) for clf in clfs]
test_scores = [clf.score(X_test, y_test) for clf in clfs]

fig, ax = plt.subplots()
ax.set_xlabel("alpha")
ax.set_ylabel("accuracy")
ax.set_title("Accuracy vs alpha for training and testing sets")
ax.plot(ccp_alphas, train_scores, marker="o", label="train", drawstyle="steps-post")
ax.plot(ccp_alphas, test_scores, marker="o", label="test", drawstyle="steps-post")
ax.legend()
plt.show()

A final Decision Tree Classifier is trained with an alpha of (???) and the best parameters found by random_search.

The trained model’s performance is evaluated on the test set using accuracy and F1-score. A classification report is also printed which provides detailed performance metrics.

In [ ]:
for alpha, imp in zip(ccp_alphas, impurities):
    print(alpha, imp)

In [ ]:
# Get the index of the best test score
idx_best_alpha = np.argmax(test_scores)

# Get the best alpha value
best_alpha = ccp_alphas[idx_best_alpha]

print("Best ccp_alpha value is: ", best_alpha)

In [ ]:
dtp = DecisionTreeClassifier(random_state=0, ccp_alpha=best_alpha, **random_search.best_params_) #choose ccp based on graphs (?)
dtp.fit(X_train, y_train)

y_test_pred = dtp.predict(X_test)

print('Train Accuracy %s' % accuracy_score(y_train, y_train_pred))
print('Train F1-score %s' % f1_score(y_train, y_train_pred, average=None))
print()

print('Test Accuracy %s' % accuracy_score(y_test, y_test_pred))
print('Test F1-score %s' % f1_score(y_test, y_test_pred, average=None))
print()

print("Accuracy:", accuracy_score(y_test, y_test_pred))
print()

print('================== Classification Report ==================')
print(classification_report(y_test, y_test_pred))

Finally, the ROC curve and Precision-Recall curve for the final model are plotted. These plots provide a comprehensive view of the model’s performance across different thresholds.

In [ ]:
from scikitplot.metrics import plot_roc
from scikitplot.metrics import plot_precision_recall

plot_roc(y_test, dtp.predict_proba(X_test))
plt.show()

plot_precision_recall(y_test, dtp.predict_proba(X_test))
plt.show()

- Micro-average ROC curve area (AUC-ROC) = 0.88: The ROC curve plots the true positive rate (TPR) against the false positive rate (FPR) at various threshold settings. The Area Under the ROC Curve (AUC-ROC) is a single-value summary of the ROC curve that ranges from 0 to 1. An AUC-ROC of 0.88 indicates a good performance of the model in distinguishing between the classes across all thresholds. A perfect classifier would have an AUC-ROC of 1.

- Micro-average Precision-Recall curve area (AUC-PR) = 0.463: The PR curve plots precision (PPV) against recall (TPR) at various threshold settings. The Area Under the PR Curve (AUC-PR) is a single-value summary of the PR curve that ranges from 0 to 1. An AUC-PR of 0.463 indicates a moderate performance of the model in terms of precision and recall. A perfect classifier would have an AUC-PR of 1.

It’s important to note that the AUC-PR can be more informative than the AUC-ROC when dealing with imbalanced datasets. However, you mentioned that your dataset is balanced, so both metrics are equally informative in this case.

In conclusion, your model shows good performance in terms of the AUC-ROC, but there is room for improvement in terms of the AUC-PR. This could indicate that while the model is good at ranking the classes correctly (as measured by the AUC-ROC), it might struggle with achieving high precision and recall simultaneously (as measured by the AUC-PR).

In summary, this code is a comprehensive demonstration of training a Decision Tree Classifier with Cost Complexity Pruning, and evaluating its performance using various metrics. It also provides visual insights into how the model’s complexity and performance vary with the pruning parameter (alpha).

The model’s performance varies significantly across different classes. Some classes (like Class 8 and Class 13) have high precision, recall, and F1-scores, indicating that the model performs well on these classes. However, other classes (like Class 7 and Class 19) have low scores, suggesting that the model struggles with these classes.

## Feature selection

Filter Methods: These methods are generally used as a preprocessing step. They use statistical measures to score each feature and select the top-scoring features. Examples include:

- Correlation Coefficient: Features with high correlation with the target variable are selected.
- Information Gain: This method is used to measure the reduction in entropy (randomness or impurity) in the target variable due to the information contained in the features.

In [ ]:
df_fs = df.copy(deep=True)

In [ ]:
# Assuming that df is your DataFrame and 'genre' is your target variable
corr_coef = df_fs.corr()

# Correlation with output variable
cor_target = abs(corr_coef["genre"])

# Sort the features by correlation
sorted_cor_target = cor_target.sort_values(ascending=False)

# Print the sorted features
print(sorted_cor_target)

The mutual information (MI) between two variables is a measure of the reduction in uncertainty (or entropy) about one variable given the knowledge of another. In the context of feature selection, MI measures the dependency between each feature and the target variable. A higher MI between a feature and the target variable means that the feature provides more information that can help in predicting the target variable, thus reducing its entropy.

In [ ]:
from sklearn.feature_selection import mutual_info_classif

X_fs = df_fs.drop('genre', axis=1)
y_fs = df_fs['genre']

# Assuming that X is your feature matrix and y is your target variable
mi = mutual_info_classif(X_fs, y_fs)

# Create a DataFrame with the scores
mi_scores = pd.DataFrame({'Feature': X_fs.columns, 'Score': mi})

mi_sorted = mi_scores.sort_values(by='Score', ascending=False)
print(mi_sorted)

In [ ]:
# Combine the two methods
combined_scores = pd.DataFrame({
    'Feature': X_fs.columns,
    'Correlation': cor_target[X_fs.columns],
    'Mutual Information': mi_scores.set_index('Feature')['Score']
})

# Handle any missing values
combined_scores = combined_scores.dropna()

# You can then sort this DataFrame by the sum of the two scores
combined_scores['Combined Score'] = (combined_scores['Correlation'] + combined_scores['Mutual Information'])/2
combined_scores = combined_scores.sort_values(by='Combined Score', ascending=False)

combined_scores

The results provided are a list of features with their correlation scores, mutual information, and a combined score. These scores are used to evaluate the importance of features in the model.

Here is an interpretation of some of the results:

- Danceability: It has a correlation of 0.30 and a mutual information score of 0.34 with the target variable. This suggests that danceability might be an important feature in the model, since it has a relatively high combined score of 0.32.
- Popularity: It has a lower correlation of 0.15, but a very high mutual information score of 0.48, which suggests that popularity might capture some complex information that is not captured by correlation alone.
- Acousticness, Energy, Valence, Instrumentalness, Loudness: These features have lower correlation and mutual information scores than danceability and popularity, which might suggest that they are less important. However, they could still be useful depending on the specific model and the problem we are trying to solve.
- Key_1, Key_10, ..., Key_3: These are categorical features (musical key of a song). They have both correlation and low mutual information, which suggests that they may not be very informative for the model.

In [ ]:
top_features = combined_scores.nlargest(5, 'Combined Score')['Feature']

X_fs_selected = X_fs[top_features]

# PARTITIONING

X_train, X_test, y_train, y_test = train_test_split(
    X_fs, y_fs, test_size=0.4, stratify=y, random_state=0
)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

from sklearn.preprocessing import StandardScaler

norm = StandardScaler()
norm.fit(X_train)

X_train_norm = norm.transform(X_train)
X_test_norm = norm.transform(X_test)

dt = DecisionTreeClassifier(**random_search.best_params_, ccp_alpha=best_alpha)
dt.fit(X_train, y_train)

y_train_pred = dt.predict(X_train)
y_test_pred = dt.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_test_pred))

print('Train Accuracy %s' % accuracy_score(y_train, y_train_pred))
print('Train F1-score %s' % f1_score(y_train, y_train_pred, average=None))
print()

print('Test Accuracy %s' % accuracy_score(y_test, y_test_pred))
print('Test F1-score %s' % f1_score(y_test, y_test_pred, average=None))

print(classification_report(y_test, y_test_pred))

In [ ]:
from sklearn.model_selection import cross_val_score
k = 10

scores = cross_val_score(dt, X_train_norm, y_train, cv=k)
print(f"Overall error estimate: {1 - scores.mean():.2f}")
print('Accuracy: %0.4f (+/- %0.2f)' % (scores.mean(), scores.std()))

In [ ]:
n_classes = len(df['genre'].unique())

lb = LabelBinarizer()
y_test_bin = lb.fit_transform(y_test)
best_y_test_pred_bin = lb.transform(y_test_pred)

y_test_pred_proba = dt.predict_proba(X_test_norm)
y_test_pred_labels = y_test_pred_proba.argmax(axis=1)

print(f"F1:{f1_score(y_test, y_test_pred_labels, labels=[1], average='micro'):.2f}")

plot_roc(y_test, y_test_pred_proba)
plt.show()

In [ ]:
# TEST

scores_EXT = cross_val_score(dt, X_testEXT_norm, y_ext, cv=10)
print('RESULTS ON EXTERNAL TEST SET:')
print(f"Overall error estimate: {1 - scores_EXT.mean():.2f}")
print('Accuracy: %0.4f (+/- %0.2f)' % (scores_EXT.mean(), scores_EXT.std()))

# TARGET: `popularity`

In [ ]:
dfpop = df.copy(deep=True) # copy of the original dataset

In [ ]:
popularity_bins = [0, 50, 80, 100]
popularity_labels = [0, 1, 2] # low, medium, high
dfpop['popularity_bin'] = pd.cut(dfpop['popularity'],
                                 bins=popularity_bins,
                                 labels=popularity_labels,
                                 include_lowest=True)
dfpop['popularity_bin'] = dfpop['popularity_bin'].astype('int')

In [ ]:
dfpop.info()

In [ ]:
# Drop the 'genre' column (target) from the DataFrame
df_without_target = dfpop.drop(['popularity_bin','popularity'], axis=1)

# Convert the DataFrame without the target column to a NumPy array
X = df_without_target.values

# Save column names
columns = df_without_target.columns.tolist()

# Convert target column to NumPy array
y = np.array(dfpop['popularity_bin'])

# Check unique values and their counts
np.unique(y, return_counts=True)

So there are 13022 songs with `low` popularity, 1956 with `medium` and 22 with `high`, according to the choosen intervals.

In [ ]:
# PARTITIONING and STANDARDIZATION

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.4, stratify=y, random_state=0
)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

from sklearn.preprocessing import StandardScaler

norm = StandardScaler()
norm.fit(X_train)

X_train_norm = norm.transform(X_train)
X_test_norm = norm.transform(X_test)

## Decision Trees

In [ ]:
%%time
# 2min

# BASE MODEL
base_dt = DecisionTreeClassifier()

base_dt.fit(X_train, y_train)

plt.figure(figsize=(20, 4), dpi=300)
plot_tree(base_dt, feature_names=columns, filled=True)
plt.show()

y_train_pred = base_dt.predict(X_train)
y_test_pred = base_dt.predict(X_test)

print('Train Accuracy %s' % accuracy_score(y_train, y_train_pred))
print('Train F1-score %s' % f1_score(y_train, y_train_pred, average=None))
print()

print('Test Accuracy %s' % accuracy_score(y_test, y_test_pred))
print('Test F1-score %s' % f1_score(y_test, y_test_pred, average=None))

print(classification_report(y_test, y_test_pred))

cf = confusion_matrix(y_test, y_test_pred)
sns.heatmap(cf, annot=True, cmap="Greens")
plt.xlabel("True")
plt.ylabel("Predicted")
plt.show()

zipped = zip(columns, base_dt.feature_importances_)
zipped = sorted(zipped, key=lambda x: x[1], reverse=True)
for col, imp in zipped:
    print(col, imp)

print(roc_auc_score(y_test, base_dt.predict_proba(X_test), multi_class="ovr", average="micro"))
scores_baseDT = cross_val_score(base_dt, X_train, y_train, cv=10)
print(f"Overall error estimate: {1 - scores_baseDT.mean():.2f}")
print('Accuracy: %0.4f (+/- %0.2f)' % (scores_baseDT.mean(), scores_baseDT.std()))

In [ ]:
range_depth = depth_param_graph(interval=range(1, 51, 1), train_X=X_train, train_y=y_train, cv=5)

In [ ]:
range_split = split_param_graph(interval=range(2, 102, 2), train_X=X_train, train_y=y_train, cv=5)

In [ ]:
range_leaf = leaf_param_graph(interval=range(1, 101, 1), train_X=X_train, train_y=y_train, cv=5)

In [ ]:
%%time
# 8min

# Your parameter list remains the same
param_list = {
    'max_depth': range_depth,
    'min_samples_split': range_split,
    'min_samples_leaf': range_leaf,
    'criterion': ['gini', 'entropy'],
    'splitter':['best','random']
}

# Create a new instance of RandomizedSearchCV
random_search = RandomizedSearchCV(
    DecisionTreeClassifier(),
    param_distributions=param_list,
    cv=RepeatedStratifiedKFold(random_state=0),
    n_jobs=-1, # number of jobs to run in parallel, -1 means using all processors
    refit=True, # allows using predict directly with this estimator
    n_iter=500 # the total size of your parameter space is the product of the number of values for each parameter; n_iter <= size (now 500)
    #verbose=2
)

random_search.fit(X_train, y_train)

dt = random_search.best_estimator_

In [ ]:
print(random_search.best_params_, random_search.best_score_)
y_train_pred = dt.predict(X_train)
y_test_pred = dt.predict(X_test)
print()
print("Accuracy:", accuracy_score(y_test, y_test_pred))
print()
print('Train Accuracy %s' % accuracy_score(y_train, y_train_pred))
print('Train F1-score %s' % f1_score(y_train, y_train_pred, average=None))
print()

print('Test Accuracy %s' % accuracy_score(y_test, y_test_pred))
print('Test F1-score %s' % f1_score(y_test, y_test_pred, average=None))
print()

print('================== Classification Report ==================')
print(classification_report(y_test, y_test_pred))
print('===========================================================')
zipped = zip(columns, dt.feature_importances_)
zipped = sorted(zipped, key=lambda x: x[1], reverse=True)
for col, imp in zipped:
    print(col, imp)

In [ ]:
from scikitplot.metrics import plot_roc
from scikitplot.metrics import plot_precision_recall

plot_roc(y_test, dt.predict_proba(X_test))
plt.show()

plot_precision_recall(y_test, dt.predict_proba(X_test))
plt.show()

### Class Weights

In [ ]:
%%time
# 4min

from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from tqdm.notebook import tqdm

# Define a range of weights
weights = [{0:1, 1:x, 2:y} for x in range(10, 110, 10) for y in range(100, 1100, 100)]

# Store the cross-validated scores for each set of weights
scores = []

# Use tqdm to show progress
for weight in tqdm(weights):
    dt = DecisionTreeClassifier(class_weight=weight)
    score = cross_val_score(dt, X_train, y_train, cv=10, scoring='f1_weighted')
    scores.append(score.mean())

# Find the weights with the highest score
optimal_weights = weights[scores.index(max(scores))]
print(optimal_weights)

In [ ]:
dt_weighted = DecisionTreeClassifier(random_state=0, **random_search.best_params_, class_weight=optimal_weights)
dt_weighted.fit(X_train, y_train)

y_train_pred = dt_weighted.predict(X_train)
y_test_pred = dt_weighted.predict(X_test)
print()
print("Accuracy:", accuracy_score(y_test, y_test_pred))
print()
print('Train Accuracy %s' % accuracy_score(y_train, y_train_pred))
print('Train F1-score %s' % f1_score(y_train, y_train_pred, average=None))
print()

print('Test Accuracy %s' % accuracy_score(y_test, y_test_pred))
print('Test F1-score %s' % f1_score(y_test, y_test_pred, average=None))
print()

print('================== Classification Report ==================')
print(classification_report(y_test, y_test_pred))
print('===========================================================')
zipped = zip(columns, dt_weighted.feature_importances_)
zipped = sorted(zipped, key=lambda x: x[1], reverse=True)
for col, imp in zipped:
    print(col, imp)

In [ ]:
plot_roc(y_test, dt_weighted.predict_proba(X_test))
plt.show()

plot_precision_recall(y_test, dt_weighted.predict_proba(X_test))
plt.show()

### ccp_alphas

In [ ]:
path = dt_weighted.cost_complexity_pruning_path(X_train, y_train)
ccp_alphas, impurities = path.ccp_alphas, path.impurities

clfs = []
for ccp_alpha in ccp_alphas:
    clf = DecisionTreeClassifier(random_state=0, ccp_alpha=ccp_alpha, **random_search.best_params_, class_weight=optimal_weights)
    clf.fit(X_train, y_train)
    clfs.append(clf)

clfs = clfs[:-1]
ccp_alphas = ccp_alphas[:-1]

node_counts = [clf.tree_.node_count for clf in clfs]
depth = [clf.tree_.max_depth for clf in clfs]

train_scores = [clf.score(X_train, y_train) for clf in clfs]
test_scores = [clf.score(X_test, y_test) for clf in clfs]

# Get the index of the best test score
idx_best_alpha = np.argmax(test_scores)

# Get the best alpha value
best_alpha = ccp_alphas[idx_best_alpha]

print("Best ccp_alpha value is: ", best_alpha)

In [ ]:
dtp = DecisionTreeClassifier(random_state=0, ccp_alpha=best_alpha, **random_search.best_params_, class_weight=optimal_weights) #choose ccp based on graphs (?)
dtp.fit(X_train, y_train)

y_test_pred = dtp.predict(X_test)

print('Train Accuracy %s' % accuracy_score(y_train, y_train_pred))
print('Train F1-score %s' % f1_score(y_train, y_train_pred, average=None))
print()

print('Test Accuracy %s' % accuracy_score(y_test, y_test_pred))
print('Test F1-score %s' % f1_score(y_test, y_test_pred, average=None))
print()

print("Accuracy:", accuracy_score(y_test, y_test_pred))
print()

print('================== Classification Report ==================')
print(classification_report(y_test, y_test_pred))

In [ ]:
plot_roc(y_test, dtp.predict_proba(X_test))
plt.show()

plot_precision_recall(y_test, dtp.predict_proba(X_test))
plt.show()

### Feature selection

In [ ]:
%%time
# ???

df_fs = dfpop.copy(deep=True)
df_fs['popularity_bin'] = df_fs['popularity_bin'].astype('int')

corr_coef = df_fs.corr()

# Correlation with output variable
cor_target = abs(corr_coef["popularity_bin"])

# Sort the features by correlation
sorted_cor_target = cor_target.sort_values(ascending=False)

from sklearn.feature_selection import mutual_info_classif

y_fs = df_fs['popularity_bin']
X_fs = df_fs.drop(['popularity', 'popularity_bin'], axis=1)

# Assuming that X is your feature matrix and y is your target variable
mi = mutual_info_classif(X_fs, y_fs)

# Create a DataFrame with the scores
mi_scores = pd.DataFrame({'Feature': X_fs.columns, 'Score': mi})

mi_sorted = mi_scores.sort_values(by='Score', ascending=False)

# Combine the two methods
combined_scores = pd.DataFrame({
    'Feature': X_fs.columns,
    'Correlation': cor_target[X_fs.columns],
    'Mutual Information': mi_scores.set_index('Feature')['Score']
})

# Handle any missing values
combined_scores = combined_scores.dropna()

# You can then sort this DataFrame by the sum of the two scores
combined_scores['Combined Score'] = (combined_scores['Correlation'] + combined_scores['Mutual Information'])/2
combined_scores = combined_scores.sort_values(by='Combined Score', ascending=False)

top_features = combined_scores.nlargest(5, 'Combined Score')['Feature']

X_fs_selected = X_fs[top_features]

# PARTITIONING

X_train, X_test, y_train, y_test = train_test_split(
    X_fs, y_fs, test_size=0.4, stratify=y, random_state=0
)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

from sklearn.preprocessing import StandardScaler

norm = StandardScaler()
norm.fit(X_train)

X_train_norm = norm.transform(X_train)
X_test_norm = norm.transform(X_test)

dt = DecisionTreeClassifier(**random_search.best_params_, ccp_alpha=best_alpha, class_weight=optimal_weights)
dt.fit(X_train, y_train)

y_train_pred = dt.predict(X_train)
y_test_pred = dt.predict(X_test)

print('Train Accuracy %s' % accuracy_score(y_train, y_train_pred))
print('Train F1-score %s' % f1_score(y_train, y_train_pred, average=None))
print()

print('Test Accuracy %s' % accuracy_score(y_test, y_test_pred))
print('Test F1-score %s' % f1_score(y_test, y_test_pred, average=None))
print()
print(classification_report(y_test, y_test_pred))
print()
scores = cross_val_score(dt, X_train_norm, y_train, cv=10)
print(f"Overall ERROR estimate: {1 - scores.mean():.2f}")
print('ACCURACY: %0.4f (+/- %0.2f)' % (scores.mean(), scores.std()))


n_classes = len(df_fs['popularity_bin'].unique())

lb = LabelBinarizer()
y_test_bin = lb.fit_transform(y_test)
best_y_test_pred_bin = lb.transform(y_test_pred)

y_test_pred_proba = dt.predict_proba(X_test_norm)
y_test_pred_labels = y_test_pred_proba.argmax(axis=1)

print(f"F1:{f1_score(y_test, y_test_pred_labels, labels=[1], average='micro'):.2f}")

plot_roc(y_test, y_test_pred_proba)
plt.show()

In [ ]:
# EXTERNAL TEST SET
test_df['popularity_bin'] = pd.cut(test_df['popularity'],
                                 bins=popularity_bins,
                                 labels=popularity_labels,
                                 include_lowest=True)
test_df['popularity_bin'] = test_df['popularity_bin'].astype('int')


y_ext = np.array(test_df['popularity_bin'])
X_testEXT = test_df.drop(['popularity_bin', 'popularity'], axis=1).values

X_testEXT_norm = norm.transform(X_testEXT)

scores_EXT = cross_val_score(dt, X_testEXT_norm, y_ext, cv=10)
print('RESULTS ON EXTERNAL TEST SET:')
print(f"Overall error estimate: {1 - scores_EXT.mean():.2f}")
print('Accuracy: %0.4f (+/- %0.2f)' % (scores_EXT.mean(), scores_EXT.std()))